In [1]:
from transformers import AutoTokenizer, AutoModel
import torch, os

MODEL_ID  = "ibm-research/MoLFormer-XL-both-10pct"
LOCAL_DIR = "/ShangGaoAIProjects/Lingge/LINCS/data/Models/MoLFormer"
os.makedirs(LOCAL_DIR, exist_ok=True)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model     = AutoModel.from_pretrained(MODEL_ID, trust_remote_code=True)

# ✅ Save in PyTorch .bin format (no safetensors)
tokenizer.save_pretrained(LOCAL_DIR)
model.save_pretrained(LOCAL_DIR, safe_serialization=False)

print("Saved to:", LOCAL_DIR)


/home/sgao30/anaconda3/envs/diffusion/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Saved to: /ShangGaoAIProjects/Lingge/LINCS/data/Models/MoLFormer


In [2]:
tokenizer = AutoTokenizer.from_pretrained(LOCAL_DIR, trust_remote_code=True)
model     = AutoModel.from_pretrained(LOCAL_DIR, trust_remote_code=True).eval()


/home/sgao30/anaconda3/envs/diffusion/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
# 3) 快速自测：给几条 SMILES 做一次前向
smiles_list = [
    "COc1ccccc1C1N(c2ccc(Cl)cc2)C(=O)C2CCCN21",   # Aspirin
    "CC(C=CC(=O)NO)=CC(C)C(=O)c1ccc(N(C)C)cc1"  # Caffeine
]

inputs = tokenizer(
    smiles_list,
    padding=True,
    truncation=True,
    return_tensors="pt"
)

with torch.no_grad():
    outputs = model(**inputs)

# 一般会有 pooler_output 可直接作为分子级 embedding（不同版本可能略有差异）
emb = getattr(outputs, "pooler_output", None)
if emb is None:
    emb = outputs.last_hidden_state[:, 0, :]  # 退化：用 CLS 作为向量
print("Emb shape:", emb.shape)


Emb shape: torch.Size([2, 768])


In [5]:
emb

tensor([[-0.0238,  0.0363,  0.6719,  ...,  0.2420,  0.5480, -0.4556],
        [ 0.2572,  0.4921,  0.8366,  ...,  0.1590, -0.1318, -0.1877]])